In [1]:
import requests
import pandas as pd
import datetime as dt
import time
import praw

In [2]:
base_url = 'https://api.pushshift.io/reddit/submission/search'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
current_time = round(time.time())

In [5]:
params = {
    'subreddit' : 'superhoops',
    'limit' : 1000,
    'until' : current_time
    
}

In [6]:
res = requests.get(base_url, params)

In [7]:
df = pd.DataFrame(res.json()['data'])
df.shape

(1000, 101)

In [8]:
def get_submission_ids(subreddit):
    submissions = []
    url = f"https://api.pushshift.io/reddit/search/submission?subreddit={subreddit}&size=200"
    start = dt.datetime(2011, 1, 1).timestamp()  
    num_posts = 0
    all_posts = []
    while True:
        start_time = time.time()
        res = requests.get(url)
        print(res.status_code)
        try:
            posts = pd.DataFrame(res.json()['data'])
            all_posts.append(posts)
            num_posts += len(posts)
            end_time = time.time()
            time_to = end_time - start_time
            total_time += time_to
        except:
            pass
        print(f"time to scrape: {time_to}; num posts: {num_posts}")
    df = pd.DataFrame(all_posts)
    return df[['id']]


In [23]:
def pull_reddit_posts(subreddit):
    """
    This function is designed to take in a set of subreddits, connect to the pushshift api,
    and pull all submissions by setting a lowerbound of choice as start_utc. Utilizes a while
    loop to get posts using requests and has checks inside for:
    1. If data is not in the response json
    2. If the last post of the most recent pull is less than the lower bounds
    Once all posts have been collected and/or the loop breaks, save all posts as a dataframe
    and save to csv in project data folder.

    """
    posts = []

    
    print("this is subreddit", subreddit)
    # save URL based on current subreddit
    url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=500"
    # set date as lower bounds, see if we can limit calls by only pulling since 2015
    start_utc = dt.datetime(2011, 1, 1).timestamp()
    print("start_utc", start_utc)
    while True:
        start_time = time.time()
        # call requests, save response
        response = requests.get(url)
        # save json as data
        data = response.json()
        # if data key not in response, break loop
        if len(data['data']) == 0:
            break
        # else, add data to posts list
        posts += data['data']
        print("posts length", len(posts))
        # pull last post in most recent data pull
        last_post = data['data'][-1]
        # save created_utc for gate and next url
        created_utc = last_post['created_utc']
        print("created_utc", created_utc)
        # current last post utc less than lower bounds, break loop
        if created_utc < start_utc:
            break
        # if loop hasn't broken, set url for next pull passing created_utc as new 'before' parameter
        url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=500&before={created_utc}"
        end_time = time.time()
        print("total time taken this loop: ", end_time-start_time)

    df = pd.DataFrame(posts)
    return df

In [24]:
df_n = pull_reddit_posts('superhoops')

this is subreddit superhoops
start_utc 1293840000.0
posts length 500
created_utc 1578825318
total time taken this loop:  2.75178599357605
posts length 1000
created_utc 1540587289
total time taken this loop:  1.6524651050567627
posts length 1500
created_utc 1504031175
total time taken this loop:  1.6351537704467773
posts length 2000
created_utc 1477517299
total time taken this loop:  2.440308094024658
posts length 2500
created_utc 1457268818
total time taken this loop:  1.9478530883789062
posts length 3000
created_utc 1441445221
total time taken this loop:  1.9085938930511475
posts length 3500
created_utc 1432035771
total time taken this loop:  2.255284070968628
posts length 4000
created_utc 1419768609
total time taken this loop:  1.8593542575836182
posts length 4500
created_utc 1409431850
total time taken this loop:  1.483976125717163
posts length 5000
created_utc 1399899486
total time taken this loop:  1.358229160308838
posts length 5499
created_utc 1389131886
total time taken this lo

In [27]:
df_n

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,thumbnail_height,top_awarded_type,hide_score,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,score,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,link_flair_type,wls,removed_by_category,author_flair_type,domain,allow_live_comments,suggested_sort,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,removed_by,distinguished,subreddit_id,link_flair_background_color,id,is_robot_indexable,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,retrieved_utc,updated_utc,utc_datetime_str,post_hint,url_overridden_by_dest,preview,link_flair_template_id,crosspost_parent_list,crosspost_parent,is_gallery,media_metadata,gallery_data,poll_data,author_cakeday,author_created_utc,edited_on,removal_reason,retrieved_on,approved_by,created,downs,likes,mod_reports,num_reports,post_categories,report_reasons,rte_mode,saved,selftext_html,ups,user_reports,brand_safe,approved_at_utc,banned_at_utc,from,from_id,from_kind
0,superhoops,Hey guys. I’m making a Queens Park Rangers Car...,t2_cu65p6m1,0.0,QPR Career mode,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,None,public,0.0,{},NaN,None,False,None,False,False,NaN,{},None,1,False,False,self,False,None,[],{},NaN,True,text,6.0,None,text,self.superhoops,False,NaN,NaN,False,True,True,False,False,[],[],False,True,False,False,None,[],NaN,None,t5_2swvo,,121vebo,True,Historical_Giraffe45,NaN,0,True,all_ads,False,False,None,/r/superhoops/comments/121vebo/qpr_career_mode/,all_ads,False,https://www.reddit.com/r/superhoops/comments/1...,2576.0,1679770444,0.0,None,False,1679770461,1679770462,2023-03-25 18:54:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,superhoops,,t2_4je6l,0.0,Queens Park Rangers C Club hospitality - REVIE...,[],r/superhoops,False,6.0,,105.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;iframe width=""267"" height=""20...",140.0,20dc4b9a-2cdb-11eb-89d3-0e959dd058df,False,"{'type': 'youtube.com', 'oembed': {'provider_u...",False,False,NaN,"{'content': '&lt;iframe width=""267"" height=""20...",Media,1,False,False,https://a.thumbs.redditmedia.com/QUFGxj8rD92me...,False,None,[],{},NaN,False,text,6.0,None,text,youtube.com,False,NaN,NaN,False,True,True,False,False,[],[],False,True,False,False,Canadian R's 🇨🇦,[],NaN,None,t5_2swvo,,11zoijt,True,peetss,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/11zoijt/queens_park_ran...,all_ads,False,https://www.youtube.com/watch?v=tamSJg2Ezfw,2575.0,1679586799,0.0,"{'type': 'youtube.com', 'oembed': {'provider_u...",False,1679586816,1679586817,2023-03-23 15:53:19,rich:video,https://www.youtube.com/watch?v=tamSJg2Ezfw,{'images': [{'source': {'url': 'https://extern...,e1a3c78c-0ae8-11e6-ac91-0e7189ea5da3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,superhoops,,t2_n02y9,0.0,"Queen's Park Rangers on Southend pier, 1924.",[],r/superhoops,False,6.0,None,102.0,NaN,True,False,dark,1.0,None,public,0.0,{},140.0,None,False,None,True,False,NaN,{},None,1,False,False,https://b.thumbs.redditmedia.com/qaMyeOQnMWLJl...,False,None,[],{},NaN,False,text,6.0,None,text,i.redd.it,False,NaN,NaN,False,True,True,False,False,[],[],False,True,False,False,None,[],NaN,None,t5_2swvo,